# Apache mod_wsgi and virtualenv deployment notes.

## Use case description

I have a little Django application in a git repo, the application is organized as explained here: https://github.com/PythonMilano/tdd-in-python

With this application is possible to reserve lab resources. It uses virtualenv to replicate in production all the stuff needed so that the application is able to run

```
requirements.txt
```

contains the information to replicate application environment.

## Deployment gotchas

Provision a ubuntu virtual machine with a deployment user, let's say is ```bookadmin```

Follow this blog to install apache and mod_wsgi http://thecodeship.com/deployment/deploy-django-apache-virtualenv-and-mod_wsgi/

## Setup application code

Upload virtualenv-x-y-z.tar.gz on the target vm and untar it in the users' home directory.

```python virtualenv-x.y.z/virtualenv.py ~/usr/labs```

```cd ~/src && git clone repo_user@repo_server:/workspace/repo_user/labs.git```

```source ~/usr/labs/bin/activate```

```pip install -r requirements.txt```

## Setup apache virtualhost

```sudo mkdir /var/www/vbook.mydomain.com```

```sudo chown bookadmin:bookadmin /var/www/vbook.mydomain.com```

```cd /etc/apache2/sites-available```

Create ```vbook.mydomain.com``` vbook is the vm name configured in the DNS. The file content is:

```
<VirtualHost vbook.mydomain.com:80>
        ServerAdmin user@mydomain.com
        ServerName vbook.mydomain.com
        ServerAlias vbook.mydomain.com
        WSGIScriptAlias / /var/www/vbook.mydomain.com/wsgi.py
        Alias /static/ /var/www/vbook.mydomain.com/static/
        <Location "/static/">
            Options -Indexes
        </Location>
</VirtualHost>
```

## Setup application for deployment

As a ```bookadmin``` user is necessary to accomplish these further steps

* Comment ```DEBUG = True``` in ```~/src/labs/labs/settings.py``` file
* Insert 'vbook.mydomain.com' into ```ALLOWED_HOSTS``` in ```~/src/labs/labs/settings.py``` file
* Update ```~/src/labs/labs/wsgi.py```
* invoke ```python manage.py collectstatic```


```wsgi.py``` content is:

```python
import os
import sys
import site

import settings

if not settings.DEBUG:
    # add site-packages for virtualenv
    site.addsitedir('/home/bookadmin/usr/labs/lib/python2.7/site-packages')

    # add project's directories
    sys.path.append('/home/bookadmin/src/labs')
    sys.path.append('/home/bookadmin/src/labs/labs')

    # Activate virtualenv
    activate_env='/home/bookadmin/usr/labs/bin/activate_this.py'
    execfile(activate_env, dict(__file__=activate_env))
    
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "labs.settings")

from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()
```

Restart apache, and that's it.

# Conclusion

* ```python manage.py runserver``` is cheap, deployment is not.
* use repositories, git or hg, because setup application code in production is easier.
* Convert all manual steps in a fabfile.py or ansible playbook and add them to your repo: codify your infrastructure!